## Building Deep Learning Models in Keras

In [1]:
import sys
!{sys.executable} -m pip install tensorflow tensorflow-datasets

You should consider upgrading via the '/home/nuls/.local/share/virtualenvs/bi_deep_learning-M-dGzlNK/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2021-09-12 17:21:13.546608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-12 17:21:13.546642: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# In keras models consist of layers
# Each layer takes a size (number of neurons), and can have a activation function and a name
layer1 = layers.Dense(2, activation='relu', name='layer1')
# An Input can be used to specify the input size 3
x = layers.Input(3, name='input')
# We can stack layers by passing it to the __call__ function of the object
model = layer1(x)
layer2 = layers.Dense(3, activation='relu', name='layer2')
model = layer2(model)

2021-09-12 17:21:16.491668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-12 17:21:16.491729: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-12 17:21:16.491748: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CE43859): /proc/driver/nvidia/version does not exist
2021-09-12 17:21:16.492025: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# A more readable way is to use Sequential models
model = keras.Sequential(
    [
        layers.Input(3, name='Input'),
        layers.Dense(2, activation='relu', name='layer1'),
        layers.Dense(3, activation='relu', name='layer2')
    ]
)

In [5]:
# Or another way would be
model = keras.Sequential()
model.add(layers.Dense(2, activation='relu', name='layer1'))
model.add(layers.Dense(3, activation='relu', name='layer2'))

In [6]:
# We can simply pass through a vector
x = tf.ones((3, 3))
y = model(x)
y

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [7]:
# We can look at it for debugging purposes with
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (3, 2)                    8         
_________________________________________________________________
layer2 (Dense)               (3, 3)                    9         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________


In [8]:
# But actually we want to train the model
# First, we need to compile it
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Possible optimizers can be found here: https://keras.io/api/optimizers/
# And loss functions here: https://keras.io/api/losses/
# For binary classification problems use: BinaryCrossentropy loss
# For classification problems use: SparseCategoricalCrossentropy loss
# For regression problems use: MeanSquaredError loss

In [9]:
# Then we can fit it
# specifying a validation data set
history = model.fit(
    x=tf.ones((3, 3)),
    y=tf.ones(3),
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(tf.ones((3, 3)), tf.ones(3)),
)

2021-09-12 17:21:17.396038: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2
1/1 [==============================] - 1s 513ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 1.0986 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 17ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 1.0986 - val_sparse_categorical_accuracy: 0.0000e+00


In [10]:
# Evaluate the model on test data
results = model.evaluate(tf.ones((3, 3)), tf.ones(3), batch_size=128)
print("test loss, test acc:", results)

1/1 [==============================] - 0s 15ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.0000e+00
test loss, test acc: [1.0986123085021973, 0.0]


In [11]:
# We can make predictions with
predictions = model.predict(tf.ones((3, 3)))

In [12]:
# For transfer learning we can 
# Remove the classification layer from the model
model.pop()

In [13]:
# Create a new model with a new classification layer
model = keras.Sequential([model, layers.Dense(10)])

In [14]:
# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).

# Alternatively
# Freeze the base model
model.trainable = False

In [15]:
# We can also build Convoluational models
model = keras.Sequential()
model.add(keras.Input(shape=(28, 28, 1)))  # 28x28 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
# to add classification layer we first need to create a one dimensional feature vector
model.add(layers.GlobalMaxPooling2D())
# Finally, we add a classification layer.
model.add(layers.Dense(10))